In [14]:
import pandas as pd
import numpy as np

In [15]:
df_routes = pd.read_csv('routes.csv')
df_airports = pd.read_csv('airports.csv')


In [16]:
display(df_routes)

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [17]:
df_routes = df_routes[['Source airport ID', 'Destination airport ID']]
df_airports = df_airports[['Airport ID','Latitude','Longitude']]

In [18]:
df_routes['Source airport ID'] = pd.to_numeric(df_routes['Source airport ID'], errors='coerce')
df_routes['Destination airport ID'] = pd.to_numeric(df_routes['Destination airport ID'], errors='coerce')
df_routes = df_routes.dropna()

df_airports['Airport ID'] = pd.to_numeric(df_airports['Airport ID'], errors='coerce')
df_airports['Latitude'] = pd.to_numeric(df_airports['Latitude'], errors='coerce')
df_airports['Longitude'] = pd.to_numeric(df_airports['Longitude'], errors='coerce')
df_airports = df_airports.dropna()

In [19]:
# Função para calcular a distância de dois pontos de latitude e longitude
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Fórmula para transformar latitude e longitude em distância (KM)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

# Merge para obter as coordenadas do aeroporto de origem
df_routes = df_routes.merge(df_airports, how='left', left_on='Source airport ID', right_on='Airport ID')
df_routes.rename(columns={'Latitude': 'Source Latitude', 'Longitude': 'Source Longitude'}, inplace=True)
df_routes.drop(['Airport ID'], axis=1, inplace=True)

# Merge para obter as coordenadas do aeroporto de destino
df_routes = df_routes.merge(df_airports, how='left', left_on='Destination airport ID', right_on='Airport ID')
df_routes.rename(columns={'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace=True)
df_routes.drop(['Airport ID'], axis=1, inplace=True)

# Calcular a distância e adicionar como nova coluna
df_routes['Distance'] = df_routes.apply(lambda row: calculate_distance(row['Source Latitude'], row['Source Longitude'], row['Destination Latitude'], row['Destination Longitude']), axis=1)

df_routes

,Source airport ID,Destination airport ID,Source Latitude,Source Longitude,Destination Latitude,Destination Longitude,Distance
0,2965.0,2990.0,43.449902,39.956600,55.606201,49.278702,1506.825604
1,2966.0,2990.0,46.283298,48.006302,55.606201,49.278702,1040.438320
2,2966.0,2962.0,46.283298,48.006302,44.225101,43.081902,448.164909
3,2968.0,2990.0,55.305801,61.503300,55.606201,49.278702,770.508500
4,2968.0,4078.0,55.305801,61.503300,55.012600,82.650703,1338.631467
...,...,...,...,...,...,...,...
67235,6334.0,3341.0,-33.058899,137.514008,-34.945000,138.531006,229.720619
67236,4029.0,2912.0,55.408798,37.906300,43.061298,74.477600,2942.819259
67237,2912.0,4029.0,43.061298,74.477600,55.408798,37.906300,2942.819259
67238,2912.0,2913.0,43.061298,74.477600,40.609001,72.793297,306.295375


In [20]:
# Eliminando todas as colunas desnecessárias.
df_routes = df_routes.drop(['Destination Latitude', 'Destination Longitude', 'Source Latitude', 'Source Longitude'], axis=1)
display(df_routes)

,Source airport ID,Destination airport ID,Distance
0,2965.0,2990.0,1506.825604
1,2966.0,2990.0,1040.438320
2,2966.0,2962.0,448.164909
3,2968.0,2990.0,770.508500
4,2968.0,4078.0,1338.631467
...,...,...,...
67235,6334.0,3341.0,229.720619
67236,4029.0,2912.0,2942.819259
67237,2912.0,4029.0,2942.819259
67238,2912.0,2913.0,306.295375


In [21]:
# Formatação do gráfico
df_routes = df_routes.sort_values(by='Distance', ascending=False)
df_routes = df_routes.dropna()
df_routes = df_routes.drop(df_routes.index[-1])
df_routes['Distance'] = df_routes['Distance'].round(2)
display(df_routes)


,Source airport ID,Destination airport ID,Distance
44962,907.0,5613.0,16082.26
44967,5613.0,907.0,16082.26
44968,5613.0,910.0,15937.11
44966,910.0,5613.0,15937.11
6807,3361.0,3670.0,13808.18
...,...,...,...
1741,9744.0,3599.0,9.15
65783,5490.0,5543.0,9.11
65815,5543.0,5490.0,9.11
38760,5567.0,5571.0,2.82


In [22]:

class Grafo:
    def __init__(self):
        self.vertices = set()
        self.arestas = {}
        self.vizinhos = {}

    # Adiciona uma aresta ao grafo. 
    # Recebe os vértices de origem e destino da aresta e o peso.
    def adiciona_aresta(self, origem, destino, peso):
        if origem not in self.vertices:
            self.vertices.add(origem)
            self.vizinhos[origem] = []
        if destino not in self.vertices:
            self.vertices.add(destino)
            self.vizinhos[destino] = []
        
        self.arestas[(origem, destino)] = peso
        self.arestas[(destino, origem)] = peso  
        self.vizinhos[origem].append((destino, peso))
        self.vizinhos[destino].append((origem, peso))

In [23]:
grafo = Grafo()
for i, row in df_routes.iterrows():
    grafo.adiciona_aresta(row['Source airport ID'], row['Destination airport ID'], row['Distance'])

In [24]:
# Algoritmo de Bellman-Ford
def bellman_ford(grafo, origem, destino):
    distancias = {vertice: float('inf') for vertice in grafo.vertices}
    antecessor = {vertice: None for vertice in grafo.vertices}
    distancias[origem] = 0

    # Relaxamento das arestas
    for _ in range(len(grafo.vertices) - 1):
        for (u, v), peso in grafo.arestas.items():
            if distancias[u] + peso < distancias[v]:
                distancias[v] = distancias[u] + peso
                antecessor[v] = u

    # Armazena o caminho mais curto
    caminho = []
    atual = destino
    while atual is not None:
        caminho.append(atual)
        atual = antecessor[atual]
    caminho = caminho[::-1]  

    if caminho[0] == origem:
        return caminho, [distancias[v] for v in caminho]
    else:
        return "Não existe caminho", []



In [25]:
# escolha qualquer Source airport ID, como origem.  
# escolha qualquer Destination airport ID, como destino.
# Você poderá checar o airport ID de cada aeroporto dentro da pasta 'airports.csv'.

display(df_routes)

# Essas rotas fornecidas embaixo sao voos diretos.
# O algoritmo fica interessante quando mistura-se dois aeroportos completamente diferentes! 

,Source airport ID,Destination airport ID,Distance
44962,907.0,5613.0,16082.26
44967,5613.0,907.0,16082.26
44968,5613.0,910.0,15937.11
44966,910.0,5613.0,15937.11
6807,3361.0,3670.0,13808.18
...,...,...,...
1741,9744.0,3599.0,9.15
65783,5490.0,5543.0,9.11
65815,5543.0,5490.0,9.11
38760,5567.0,5571.0,2.82


In [26]:
# O código vai receber duas entradas.
# A primeira será o ID de origem.
# A segunda será o ID de destino.
while True:
    origem = int(input('ID do aeroporto de origem: '))
    if (df_routes['Source airport ID'] == origem).any():
        break
    else:
        print('Esse ID do aeroporto não existe. Digite outro')
while True:
    destino = int(input('ID do aeroporto de destino: '))
    if (df_routes['Destination airport ID'] == destino).any():
        break
    else:
        print('Esse ID do aeroporto não existe. Digite outro')

caminho, distancias = bellman_ford(grafo, origem, destino)
print("Caminho:", caminho)
print("Distâncias:", distancias)

# O algoritmo mostrará a rota entre os aeroportos (caminho).
# O algoritmo Bellman-Ford encontrará o menor caminho entre esses dois aeroportos em KM! 
# A cada mudança de aeroporto, atualiza-se a distância total entre o aeroporto de origem e o aeroporto de destino.

Caminho: [2.0, 3]
Distâncias: [0, 179.04]
